In [8]:
import cfbd
import numpy as np
import pandas as pd

In [9]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'Zg3HzWsx+jNtEVi9kgZvsvtOZPy/ZbIakgsgc83IiQ0XjKludC/gEdDSWpg2CcgP'
configuration.api_key_prefix['Authorization'] = 'Bearer'
api_config = cfbd.ApiClient(configuration)
teams_api = cfbd.TeamsApi(api_config)
rankings_api = cfbd.RankingsApi(api_config)
games_api = cfbd.GamesApi(api_config)
stats_api = cfbd.StatsApi(api_config)
betting_api = cfbd.BettingApi(api_config)
metrics_api = cfbd.MetricsApi(api_config)
ratings_api = cfbd.RatingsApi(api_config)
rectruiting_api = cfbd.RecruitingApi(api_config)
conferences_api = cfbd.ConferencesApi(api_config)

In [14]:
teams = [team for team in teams_api.get_teams() if team.classification == 'fbs']

In [4]:
def get_rse():
    record = games_api.get_team_records(year=2022)
    stats = stats_api.get_team_season_stats(year=2022)
    elo = ratings_api.get_elo_ratings(year=2022)
    return record, stats, elo
def get_stats_list():
    l = []
    for x in range(2004,2022):
        k = stats_api.get_team_season_stats(year=x, team='Michigan')
        k_tot = sorted([l.stat_name for l in k])
        l.append(k_tot)
    return list(set.intersection(*map(set,l)))
def get_game_data(elo, home_team, away_team):  
    home_elo = 0
    away_elo = 0
    for team in elo:
        if team.team==home_team:
            home_elo = team.elo
        if team.team==away_team:
            away_elo = team.elo
    home_points = 0
    away_points = 0
    home_team = home_team
    away_team = away_team
    game_id = 0
    line = [game_id, home_team, away_team, home_points, away_points, home_elo, away_elo]
    header = ['game_id', 'home_team', 'away_team', 'home_points', 'away_points', 'home_elo', 'away_elo']
    return line, header
def get_talent_data(talent, home_team, away_team):
    header = ['home_talent', 'away_talent']
    line = [None, None]
    for team in talent:
        if team.school == home_team:
            line[0] = team.talent
        if team.school == away_team:
            line[1] = team.talent
    return line, header
def get_record_data(record, home_team, away_team):
    for school in record:
        if school.team == home_team:
            home_record = school
    for school in record:
        if school.team == away_team:
            away_record = school
    home_games = home_record.total.games
    home_wp = home_record.total.wins/home_record.total.games
    away_games = away_record.total.games
    away_wp = away_record.total.wins/away_record.total.games
    line = [home_games, home_wp, away_games, away_wp]
    header = ['home_games', 'home_wp', 'away_games', 'away_wp'] 
    return line, header
def get_stat_data(stats, home_team, away_team, stats_list):
    line = []
    header = []
    home_season_stats = []
    away_season_stats = []
    for stat in stats:
        if stat.team == home_team:
            if stat.stat_name == 'games':
                continue
            home_season_stats.append(stat)
        if stat.team == away_team:
            if stat.stat_name == 'games':
                continue
            away_season_stats.append(stat)
    h = {}
    a = {}
    if len(home_season_stats)<27 or len(away_season_stats)<27:
        return False, False
    for i,stat in enumerate(home_season_stats):
        if stat.stat_name in stats_list:
            h[stat.stat_name] = stat.stat_value
    for i,stat in enumerate(away_season_stats):
        if stat.stat_name in stats_list:
            a[stat.stat_name] = stat.stat_value
    if len(h)!=26:
        h['puntReturnYards']=0
        h['puntReturns']=0
        h['puntReturnTDs']=0
    if len(a)!=26:
        a['puntReturnYards']=0
        a['puntReturns']=0
        a['puntReturnTDs']=0
    for key,value in sorted(h.items()):
        header.append('home_'+key)
        line.append(value)
    for key,value in sorted(a.items()):
        header.append('away_'+key)
        line.append(value)
    return line, header
def get_metrics_data(year, home_team, away_team):
    line = []
    header = []
    home_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=home_team)
    away_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=away_team)
    if len(home_advanced_season_stats)==0 or len(away_advanced_season_stats)==0:
        return False, False
    home_advanced_season_stats = home_advanced_season_stats[0]
    away_advanced_season_stats = away_advanced_season_stats[0]
    hass = home_advanced_season_stats.to_dict()
    aass = away_advanced_season_stats.to_dict()
    
    new_line = []
    new_header = []
    side = ['defense','offense']
    for s in side:
        for key,value in sorted(hass[s].items()):
            if type(value) != dict:
                new_header.append("home_"+s+"_"+key)
                new_line.append(value)
            else:
                for key2,value2 in sorted(hass[s][key].items()):
                    new_header.append("home_"+s+"_"+key+"_"+key2)
                    new_line.append(value2)
    line.extend(new_line)
    header.extend(new_header)
    
    new_line = []
    new_header = []
    side = ['defense','offense']
    for s in side:
        for key,value in sorted(aass[s].items()):
            if type(value) != dict:
                new_header.append("away_"+s+"_"+key)
                new_line.append(value)
            else:
                for key2,value2 in sorted(aass[s][key].items()):
                    new_header.append("away_"+s+"_"+key+"_"+key2)
                    new_line.append(value2)
    line.extend(new_line)
    header.extend(new_header)
    return line, header

In [5]:
def make_df(record, stats, elo, stats_list, home_team, away_team):
    curr_line = []
    curr_header = []
    #add game data
    game_line, game_header = get_game_data(elo, home_team, away_team)
    curr_line.extend(game_line)
    curr_header.extend(game_header)

    #add record data
    record_line, record_header = get_record_data(record, home_team, away_team)
    curr_line.extend(record_line)
    curr_header.extend(record_header)

    #add season stat data
    stat_line, stat_header = get_stat_data(stats, home_team, away_team, stats_list)
    curr_line.extend(stat_line)
    curr_header.extend(stat_header)

    #add advanced metrics data
    metrics_line, metrics_header = get_metrics_data(2022, home_team, away_team)
    curr_line.extend(metrics_line)
    curr_header.extend(metrics_header)
    df = pd.DataFrame([curr_line], columns=curr_header)
    cols = list(df.columns)
    nc = []
    c = []
    for i,x in enumerate(cols):
        if i<11:
            nc.append(x)
        if 'total' in x:
            c.append(x)
        if 'drives' in x:
            c.append(x)
        if 'rate' in x:
            nc.append(x)
        if 'average' in x:
            nc.append(x)
        if 'explosiveness' in x:
            nc.append(x)
    new_cols = []
    for x in cols:
        if x not in nc and x not in c:
            new_cols.append(x)
    c.extend(new_cols[:50])
    new_cols = new_cols[50:]
    ac = 0
    hc = 0
    for i,x in enumerate(new_cols):
        if 'home' in x:
            hc+=1
        if 'away' in x:
            ac+=1
    for i,x in enumerate(new_cols):
        if 'havoc' in x:
            nc.append(x)
        if '_defense_plays' in x:
            c.append(x)
        if '_offense_plays' in x:
            c.append(x)
    for x in nc:
        if x in new_cols:
            new_cols.remove(x)
    for x in c:
        if x in new_cols:
            new_cols.remove(x)
    for i,x in enumerate(new_cols):
        nc.append(x)
    new_cols = []
    for x in c:
        if 'home' in x:
            df[x] = df[x]/df['home_games']
        if 'away' in x:
            df[x] = df[x]/df['away_games']
    df_features = df.copy()
    df_labels = pd.concat([df_features.pop(x) for x in ['home_points', 'away_points']], axis=1)
    #Remove Features
    removal_list = ['game_id', 'home_games', 'away_games', 'home_penalties', 'away_penalties',
                    'home_penaltyYards', 'away_penaltyYards','home_defense_havoc_front_seven', 
                    'away_defense_havoc_front_seven', 'home_defense_passing_plays_total_ppa', 
                    'away_defense_passing_plays_total_ppa', 'home_offense_havoc_front_seven', 
                    'away_offense_havoc_front_seven']
    unused = pd.concat([df_features.pop(x) for x in removal_list], axis=1)
#     df_features = np.array(df_features)
#     df_labels = np.array(df_labels)
    return df_features

In [10]:
record, stats, elo = get_rse()
stats_list = get_stats_list()

In [11]:
out = make_df(record, stats, elo, stats_list, 'Ohio State', 'Michigan')
out.head()

,home_team,away_team,home_elo,away_elo,home_wp,away_wp,home_firstDowns,home_fourthDownConversions,home_fourthDowns,home_fumblesLost,...,away_offense_second_level_yards,away_offense_second_level_yards_total,away_offense_standard_downs_explosiveness,away_offense_standard_downs_ppa,away_offense_standard_downs_rate,away_offense_standard_downs_success_rate,away_offense_stuff_rate,away_offense_success_rate,away_offense_total_opportunies,away_offense_total_ppa
0,Ohio State,Michigan,2318.0,2037.0,1.0,1.0,25.142857,1.285714,1.285714,0.428571,...,1.321678,54.0,0.991198,0.325393,0.76569,0.592896,0.157343,0.550209,7.285714,24.384068


In [384]:
out = make_df(record, stats, elo, stats_list, 'Ohio State', 'Michigan')
headers = out.columns
out_numpy = np.array(out)[0]
for i,x in enumerate(headers):
    if 'away' in x:
        out_numpy[i] = None

In [12]:
def make_home(record, stats, elo, stats_list, home_team, away_team):
    out = make_df(record, stats, elo, stats_list, home_team, away_team)
    headers = out.columns
    out_numpy = np.array(out)[0]
    for i,x in enumerate(headers):
        if 'away' in x:
            out_numpy[i] = None
    home = [out_numpy[0]]
    home.extend([out_numpy[2]])
    home.extend([out_numpy[4]])
    home.extend(out_numpy[6:30])
    home.extend(out_numpy[54:129])
    home_header = [headers[0]]
    home_header.extend([headers[2]])
    home_header.extend([headers[4]])
    home_header.extend(headers[6:30])
    home_header.extend(headers[54:129])
    return home, home_header

In [16]:
lines = []
for i, team in enumerate(teams):
    line, header = make_home(record, stats, elo, stats_list, team.school, 'Michigan')
    if i==0:
        lines.append(header)
    lines.append(line)

In [17]:
for x in lines:
    if len(x)!=202:
        print(len(x))

132
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102


In [400]:
import csv
with open("team_attributes.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(lines)
f.close()